## Ошибки в исходных файлах и компоненты рабочего ПО

Что нужно сделать
У нас есть набор исходников. Необходимо выполнить git clone.

Просмотрите конфиг-файлы, dockerfile, docker-compose.yml и найдите все ошибки. Например, в dockerfile в качестве исходника для образа может не быть слоя FROM nginx:alpine/, в других dockerfile может отсутствовать volume /data, порт 6379, пользователи.

В папке web в dockerfile задайте источник python:2.7-stretch, пользователя root, порт 9000.
Подсказка: в большинстве случаев в месте, где надо что-то добавить, стоит коммент через #.
Внесите правки в docker-compose.yml:
в блок services определите сети — frontend и backend;
в блок volume добавьте путь ./web/cron/crontab_todo:/etc/crontab;
в блок consumers добавьте порт 9000;в блок redis build добавьте ./redis и порт 6379.
Когда вы подумаете, что все исправления сделаны, выполните $ docker-compose up.
Если всё сделано корректно, то запустится два экземпляра Redis.

Пришлите скриншот или текст всех dockerfile в проекте, docker-compose.
Пришлите скриншот вывода команды docker ps.


In [148]:
import os
import shutil
import subprocess
from git import Repo

In [149]:
def clone_git_repo(repo_url, parent_folder, repo_name):
    os.chdir(
        "/Users/andrejizvarin/Documents/___project/NoSQL_Big_Data/"
    )  # Переход в корневую директорию

    clone_folder = os.path.join(parent_folder, repo_name)
    # Если директория существует, удаляем её
    if os.path.isdir(clone_folder):
        shutil.rmtree(clone_folder)
        print(f'Папка "{clone_folder}" была удалена.')

    # Клонируем репозиторий
    Repo.clone_from(repo_url, clone_folder)
    return f"Репозиторий успешно склонирован в {clone_folder}"


# URL репозитория для клонирования
repo_url = "https://github.com/darkbenladan/docker-nginx-python2-db-redis-homework.git"
# Имя репозитория
repo_name = "docker-nginx-python2-db-redis-homework"
# Путь к родительской папке, куда будет склонирован репозиторий
parent_folder = "data"

# Клонируем репозиторий заново
result = clone_git_repo(repo_url, parent_folder, repo_name)
print(result)

Репозиторий успешно склонирован в data/docker-nginx-python2-db-redis-homework


In [150]:
def rewrite_file(filepath, content):
    """Перезаписываем файл с заданным содержимым."""
    try:
        with open(filepath, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Файл {filepath} успешно перезаписан.")
    except Exception as e:
        print(f"Ошибка при работе с файлом {filepath}: {e}")


def main():
    base_path = "data/docker-nginx-python2-db-redis-homework"

    # Полная перезапись Dockerfile в каталоге web
    web_dockerfile_path = os.path.join(base_path, "web/Dockerfile")
    web_dockerfile_content = """FROM python:3.9-buster
RUN apt-get update -yqq && \
    apt-get install -y python3-dev gcc libpcre3 libpcre3-dev binutils libproj-dev gdal-bin
RUN pip install uwsgi
WORKDIR /var/www/web
COPY . /var/www/web
RUN pip install -r requirements.txt
RUN mkdir -p logs && chown -R www-data:www-data /var/www/web
COPY uwsgi.ini /etc/uwsgi/
CMD ["uwsgi", "--ini", "/etc/uwsgi/uwsgi.ini"]
EXPOSE 9000"""
    rewrite_file(web_dockerfile_path, web_dockerfile_content)

    # Полная перезапись Dockerfile в каталоге mysql
    mysql_dockerfile_path = os.path.join(base_path, "mysql/Dockerfile")
    mysql_dockerfile_content = """FROM mysql:latest
ARG TZ=UTC
ENV TZ ${TZ}
RUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone && chown -R mysql:root /var/lib/mysql/
RUN apt-get update && apt-get install -y git
WORKDIR /opt
RUN git clone https://github.com/darkbenladan/docker-nginx-python2-db-redis-homework.git
COPY my.cnf /etc/mysql/conf.d/my.cnf
CMD ["mysqld"]
EXPOSE 3306"""
    rewrite_file(mysql_dockerfile_path, mysql_dockerfile_content)

    # Полная перезапись Dockerfile в каталоге redis
    redis_dockerfile_path = os.path.join(base_path, "redis/Dockerfile")
    redis_dockerfile_content = """FROM redis:6.2.6
RUN mkdir -p /etc/redis
COPY redis.conf /etc/redis/redis.conf
VOLUME /data
EXPOSE 6379
CMD ["redis-server", "/etc/redis/redis.conf"]"""
    rewrite_file(redis_dockerfile_path, redis_dockerfile_content)

    # Полная перезапись Dockerfile в каталоге consumers
    consumers_dockerfile_path = os.path.join(base_path, "consumers/Dockerfile")
    consumers_dockerfile_content = """FROM python:3.9-buster
RUN apt-get update && apt-get install -y supervisor
RUN mkdir -p /var/www/todo/logs
COPY ./supervisor/supervisord.conf /etc/supervisor/supervisord.conf
COPY ./supervisor/todo.conf /etc/supervisor/conf.d/todo.conf
ADD https://raw.githubusercontent.com/vishnubob/wait-for-it/master/wait-for-it.sh /usr/local/bin/wait-for-it.sh
RUN chmod +x /usr/local/bin/wait-for-it.sh
CMD ["/usr/local/bin/wait-for-it.sh", "db:3306", "--strict", "--timeout=30", "--", "/usr/bin/supervisord", "-c", "/etc/supervisor/supervisord.conf"]
EXPOSE 9000"""
    rewrite_file(consumers_dockerfile_path, consumers_dockerfile_content)

    # Полная перезапись Dockerfile в каталоге nginx
    nginx_dockerfile_path = os.path.join(base_path, "nginx/Dockerfile")
    nginx_dockerfile_content = """FROM nginx:stable-alpine
COPY nginx.conf /etc/nginx/nginx.conf
ARG CHANGE_SOURCE=false
RUN if [ "$CHANGE_SOURCE" = "true" ]; then sed -i 's/dl-cdn.alpinelinux.org/mirrors.aliyun.com/' /etc/apk/repositories; fi
RUN apk update && apk upgrade && apk add --no-cache openssl bash
RUN if [ $(grep -c www-data /etc/group) = 0 ]; then adduser -D -H -u 1000 -s /bin/bash www-data; else adduser -D -H -u 1000 -s /bin/bash www-data -G www-data; fi
ADD ./startup.sh /opt/startup.sh
RUN sed -i 's/\r//g' /opt/startup.sh
RUN chmod +x /opt/startup.sh
ADD https://github.com/vishnubob/wait-for-it/raw/master/wait-for-it.sh /usr/local/bin/wait-for-it.sh
RUN chmod +x /usr/local/bin/wait-for-it.sh
CMD ["/usr/local/bin/wait-for-it.sh", "todo:9000", "--strict", "--timeout=300", "--", "/bin/bash", "/opt/startup.sh"]
EXPOSE 80 443"""
    rewrite_file(nginx_dockerfile_path, nginx_dockerfile_content)

    # Полная перезапись docker-compose.yml
    docker_compose_path = os.path.join(base_path, "docker-compose.yml")
    docker_compose_content = """version: '3.8'

services:
  nginx_todo:
    build:
      context: ./nginx
    volumes:
      - ${APP_CODE_TODO_PATH_HOST}:${APP_CODE_TODO_PATH_CONTAINER}
      - ${NGINX_TODO_HOST_LOG_PATH}:/var/log/nginx
      - ${NGINX_TODO_SITE_PATH}:/etc/nginx/sites-available/todo.conf
      - ${NGINX_SSL_PATH}:/etc/nginx/ssl
    ports:
      - "${NGINX_TODO_HOST_HTTP_PORT}:80"
      - "${NGINX_TODO_HOST_HTTPS_PORT}:443"
    depends_on:
      - todo
    logging:
      driver: none

  todo:
    build:
      context: ./web
    volumes:
      - ${APP_CODE_TODO_PATH_HOST}:${APP_CODE_TODO_PATH_CONTAINER}
    expose:
      - "9000"
    depends_on:
      - db
      - redis
    logging:
      driver: none

  consumers:
    build:
      context: ./consumers
    volumes:
      - ./consumers/supervisor/todo.conf:/etc/supervisor/conf.d/todo.conf
      - ${APP_CODE_TODO_PATH_HOST}:${APP_CODE_TODO_PATH_CONTAINER}
    expose:
      - "9000"
    depends_on:
      - db
      - redis
    logging:
      driver: none

  db:
    image: mysql:latest
    environment:
      - MYSQL_DATABASE=${MYSQL_DATABASE}
      - MYSQL_USER=${MYSQL_USER}
      - MYSQL_PASSWORD=${MYSQL_PASSWORD}
      - MYSQL_ROOT_PASSWORD=${MYSQL_ROOT_PASSWORD}
    volumes:
      - ${DATA_PATH_HOST}/mysql:/var/lib/mysql
    ports:
      - "${MYSQL_PORT}:3306"
    logging:
      driver: none

  redis:
    image: redis:latest
    volumes:
      - ${DATA_PATH_HOST}/redis:/data
    ports:
      - "${REDIS_PORT}:6379"
    logging:
      driver: none

networks:
  default:
    driver: bridge"""
    rewrite_file(docker_compose_path, docker_compose_content)

    # Полная перезапись startup.sh
    startup_sh_path = os.path.join(base_path, "startup.sh")
    startup_sh_content = """#!/bin/bash
/wait-for-it.sh db:3306 --timeout=30 --strict -- echo "Database is up."
/wait-for-it.sh todo:9000 --timeout=30 --strict -- echo "Todo service is up."
if [ ! -d "/var/www/web/logs" ]; then
  mkdir -p /var/www/web/logs
  echo "Created logs directory."
fi
/usr/bin/supervisord -c /var/www/consumers/supervisor/supervisord.conf"""
    rewrite_file(startup_sh_path, startup_sh_content)

    # Полная перезапись uwsgi.ini в папке web
    uwsgi_path = os.path.join(base_path, "web/uwsgi.ini")
    uwsgi_content = """[uwsgi]
module = wsgi:application
pythonpath = /path/to/your/project/web
socket = :9000
chown-socket = www-data:www-data
chmod-socket = 660
master = true
processes = 4
threads = 2
vacuum = true
hook-master-start = unix_signal:15 gracefully_kill_them_all
need-app = true
die-on-term = true
plugins-dir = /usr/lib/uwsgi/plugins
plugin = python
py-autoreload = 1
show-config = true
cheaper = 2
processes = 16"""
    rewrite_file(uwsgi_path, uwsgi_content)

    # Создание файла wsgi.py в папке проекта web
    wsgi_path = os.path.join(base_path, "web/wsgi.py")
    wsgi_content = """def application(environ, start_response):
    status = '200 OK'
    headers = [('Content-type', 'text/plain; charset=utf-8')]
    start_response(status, headers)
    return [b"Hello World! This is a simple WSGI Application."]"""
    rewrite_file(wsgi_path, wsgi_content)

    # Полная перезапись todo.conf consumers/supervisor/
    todo_conf_path = os.path.join(
        base_path,
        "consumers/supervisor/todo.conf",
    )
    todo_conf_content = """[program:send-notification]
command=celery worker -A todo --loglevel=info
process_name=%(program_name)s_%(process_num)02d
numprocs=2
directory=/var/www/todo
autostart=true
autorestart=true"""
    rewrite_file(todo_conf_path, todo_conf_content)

    # Полная перезапись supervisord.conf consumers/supervisor/
    supervisord_conf_path = os.path.join(
        base_path,
        "consumers/supervisor/supervisord.conf",
    )
    supervisord_conf_content = """[unix_http_server]
file=/var/run/supervisor.sock
chmod=0770
chown=www-data:www-data

[supervisord]
nodaemon=true
logfile_maxbytes=0
logfile_backups=0

[rpcinterface:supervisor]
supervisor.rpcinterface_factory=supervisor.rpcinterface:make_main_rpcinterface

[supervisorctl]
serverurl=unix:///var/run/supervisor.sock

[inet_http_server]
port = 9000
username = www-data
password = www-data

[include]
files=/etc/supervisor/conf.d/*.conf"""
    rewrite_file(supervisord_conf_path, supervisord_conf_content)


if __name__ == "__main__":
    main()

Файл data/docker-nginx-python2-db-redis-homework/web/Dockerfile успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/mysql/Dockerfile успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/redis/Dockerfile успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/consumers/Dockerfile успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/nginx/Dockerfile успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/docker-compose.yml успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/startup.sh успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/web/uwsgi.ini успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/web/wsgi.py успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/consumers/supervisor/todo.conf успешно перезаписан.
Файл data/docker-nginx-python2-db-redis-homework/consumers/supervisor/supervisord.conf успешно перезаписан.


In [151]:
# sudo chmod +rw /Users/andrejizvarin/.docker/buildx/current
# sudo chmod -R 777 /Users/andrejizvarin/Documents/___project/NoSQL_Big_Data/data/docker-nginx-python2-db-redis-homework

In [152]:
# def print_tree(directory, indent=""):
#     items = os.listdir(directory)
#     for item in sorted(items):
#         path = os.path.join(directory, item)
#         if os.path.isdir(path):
#             print(f"{indent}{item}/")
#             print_tree(path, indent + "    ")
#         else:
#             print(f"{indent}{item}")


# directory_path = "/Users/andrejizvarin/Documents/___project/NoSQL_Big_Data/data/docker-nginx-python2-db-redis-homework"
# print_tree(directory_path)

In [153]:
def docker_compose_up(base_path):
    try:
        # Переход в директорию с файлом docker-compose.yml
        os.chdir(base_path)
        # Выполнение команды docker-compose up
        result = subprocess.run(["docker-compose", "up", "-d"], check=True)
        print("Docker-compose успешно запущен.")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка выполнения docker-compose up: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")


def main():
    base_path = "data/docker-nginx-python2-db-redis-homework"
    docker_compose_up(base_path)


if __name__ == "__main__":
    main()

time="2024-04-26T23:54:22+03:00" level=warning msg="/Users/andrejizvarin/Documents/___project/NoSQL_Big_Data/data/docker-nginx-python2-db-redis-homework/docker-compose.yml: `version` is obsolete"
 redis Pulling 
 db Pulling 
 c6a0976a2dbe Already exists 
 8dd4f8e415ca Already exists 
 6e01a6ece3af Already exists 
 6cfdeffd9140 Already exists 
 73fed55ee93c Already exists 
 83404f4e4847 Already exists 
 aad53405df78 Already exists 
 d9c5f6f4cc6e Already exists 
 e04d803ff9c7 Already exists 
 f06a309d43da Already exists 
 22d97f6a5d13 Already exists 
 c7b117eba408 Pulling fs layer 
 3549e2a23473 Pulling fs layer 
 ccadc22b76d5 Pulling fs layer 
 d4ed6d335745 Pulling fs layer 
 00c30f88f8b5 Pulling fs layer 
 4f4fb700ef54 Pulling fs layer 
 46ea6f8350c2 Pulling fs layer 
 d4ed6d335745 Waiting 
 4f4fb700ef54 Waiting 
 46ea6f8350c2 Waiting 
 00c30f88f8b5 Waiting 
 c7b117eba408 Downloading [================================>                  ]     720B/1.103kB
 c7b117eba408 Downloading [=====

#0 building with "desktop-linux" instance using docker driver

#1 [todo internal] load build definition from Dockerfile
#1 transferring dockerfile: 425B done
#1 DONE 0.0s

#2 [consumers internal] load build definition from Dockerfile
#2 transferring dockerfile: 590B done
#2 DONE 0.0s

#3 [consumers auth] library/python:pull token for registry-1.docker.io
#3 DONE 0.0s

#4 [todo internal] load metadata for docker.io/library/python:3.9-buster
#4 DONE 1.0s

#5 [todo internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [consumers internal] load .dockerignore
#6 transferring context: 2B done
#6 DONE 0.0s

#7 [todo internal] load build context
#7 transferring context: 2.12kB done
#7 DONE 0.0s

#8 [todo 3/8] RUN pip install uwsgi
#8 CACHED

#9 [todo 2/8] RUN apt-get update -yqq &&     apt-get install -y python3-dev gcc libpcre3 libpcre3-dev binutils libproj-dev gdal-bin
#9 CACHED

#10 [todo 4/8] WORKDIR /var/www/web
#10 CACHED

#11 [todo 6/8] RUN pip install -r requir

 Container docker-nginx-python2-db-redis-homework-redis-1  Creating
 Container docker-nginx-python2-db-redis-homework-db-1  Creating
 Container docker-nginx-python2-db-redis-homework-redis-1  Created
 Container docker-nginx-python2-db-redis-homework-db-1  Created
 Container docker-nginx-python2-db-redis-homework-todo-1  Creating
 Container docker-nginx-python2-db-redis-homework-consumers-1  Creating
 Container docker-nginx-python2-db-redis-homework-todo-1  Created
 Container docker-nginx-python2-db-redis-homework-nginx_todo-1  Creating
 Container docker-nginx-python2-db-redis-homework-consumers-1  Created
 Container docker-nginx-python2-db-redis-homework-nginx_todo-1  Created
 Container docker-nginx-python2-db-redis-homework-redis-1  Starting
 Container docker-nginx-python2-db-redis-homework-db-1  Starting
 Container docker-nginx-python2-db-redis-homework-db-1  Started
 Container docker-nginx-python2-db-redis-homework-redis-1  Started
 Container docker-nginx-python2-db-redis-homework-t

Docker-compose успешно запущен.


 Container docker-nginx-python2-db-redis-homework-nginx_todo-1  Started


In [154]:
# def run_docker_commands():
#     try:
#         print("Останавливаем и удаляем текущие контейнеры...")
#         subprocess.run(["docker-compose", "down"], check=True)

#         print("Пересобираем и запускаем контейнеры...")
#         subprocess.run(["docker-compose", "up", "--build", "-d"], check=True)

#         print("Контейнеры перезапущены успешно!")

#     except subprocess.CalledProcessError as e:
#         print(f"Произошла ошибка при выполнении команд docker-compose: {e}")


# if __name__ == "__main__":
#     run_docker_commands()

In [155]:
def show_docker_containers():
    print("Запущенные контейнеры:")
    subprocess.run(["docker", "ps"], check=True)


if __name__ == "__main__":
    show_docker_containers()

Запущенные контейнеры:
CONTAINER ID   IMAGE                                               COMMAND                  CREATED         STATUS         PORTS                                         NAMES
c3ec9ad33647   docker-nginx-python2-db-redis-homework-nginx_todo   "/docker-entrypoint.…"   7 minutes ago   Up 7 minutes   0.0.0.0:8081->80/tcp, 0.0.0.0:4444->443/tcp   docker-nginx-python2-db-redis-homework-nginx_todo-1
d291a0ca99e1   docker-nginx-python2-db-redis-homework-consumers    "/usr/local/bin/wait…"   7 minutes ago   Up 7 minutes   9000/tcp                                      docker-nginx-python2-db-redis-homework-consumers-1
136a9f136074   docker-nginx-python2-db-redis-homework-todo         "uwsgi --ini /etc/uw…"   7 minutes ago   Up 7 minutes   9000/tcp                                      docker-nginx-python2-db-redis-homework-todo-1
9e373b32526e   redis:latest                                        "docker-entrypoint.s…"   7 minutes ago   Up 7 minutes   0.0.0.0:6379->6379/tcp 